@author: Gediyon M. Girma

Off-policy expected SARSA with a greedy control policy for MoutainCar environmnet

In [10]:
import gym
import time
import itertools
import random
import numpy as np
env = gym.make('MountainCar-v0')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [11]:
# discretization of the observation space

bins = (30, 30)

def discretize_observation(observation):
  """
  Discretizes the observation space.

  Args:
    observation: The observation to discretize.

  Returns:
    The discretized observation.
  """

  low = env.observation_space.low
  high = env.observation_space.high

  return tuple(np.digitize(observation[i], np.linspace(low[i], high[i], bins[i] + 1)) - 1 for i in range(2))

In [12]:
#initialization
# the off-policy SARSA with greedy control policy
alpha = 0.1
gamma = 1
epsilon = 0.1

# formaulate the state space with every combination of the discritsized elements of the states
states = itertools.product(np.arange(bins[0]), np.arange(bins[1]))

q={}
policy = {}
b = {}  # behavioural policy


for state in states:
  q[state] = np.zeros((env.action_space.n)) # initialize the action-value function

  # initializing the epsilon-greedy policy with respect to the action-value function
  b[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n)
  b[state][np.random.randint(3)] += 1-epsilon



In [13]:


episodes = 5e4
start_timer = time.time()
episode = 1
while episode < episodes:

  # reset the environment
  obs = env.reset()
  state = discretize_observation(obs) # discretize the observation

  for t in range(1,201):
    action = np.random.choice(np.arange(env.action_space.n), p = b[state]) # select an action
    obs, reward, done, info = env.step(action)
    next_state = discretize_observation(obs) # next state

    # update the action-value function
    q[state][action] += alpha * (reward + gamma * np.max(q[next_state]) - q[state][action]) # update the action-value function

    # update the policy based on the action-value function
    best_action = np.argmax(q[state]) # selecting the best action based on the action-value function
    b[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n) # resetting the policy for the given state
    b[state][best_action] += 1 - epsilon # setting the probability of the best action to 1


    state = next_state

    # check if the episode is done
    if done:
      break


  episode += 1
  if episode % 10000 == 0:
    end_timer = time.time()
    timer = end_timer - start_timer
    elapsed_time_struct = time.gmtime(timer)
    formatted_time = time.strftime("%H:%M:%S", elapsed_time_struct)
    print("Episode: ",episode, " time: ", formatted_time)

  if episode == episodes:
    print("done!")


# synthesyzing a policy from the final action-value function
for state in states:
  best_action = np.argmax(q[state])
  policy[state] = np.zeros((env.action_space.n)) # initialize the policy
  policy[state][best_action] += 1



/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:  10000  time:  00:06:05
Episode:  20000  time:  00:11:25
Episode:  30000  time:  00:16:31
Episode:  40000  time:  00:21:32
Episode:  50000  time:  00:26:29
done!


In [17]:
#test
steps_to_solution = []

for j in range(100):
  obs = env.reset()
  state = discretize_observation(obs)
  for i in range(1,50000):
    action = np.argmax(q[state])
    # print("step",i, "reward: ", reward," state: ", state," action=",action)
    obs, reward, done, info = env.step(action)
    state = discretize_observation(obs)

    if done:
      #print ("done")
      steps_to_solution.append(i)
      break
print(steps_to_solution)
avg_step = np.mean(steps_to_solution)
print("Average steps to solution: ",avg_step)

# ipythondisplay.clear_output(wait=True)
env.close()

[146, 175, 147, 144, 178, 145, 149, 185, 149, 147, 145, 175, 141, 200, 146, 146, 147, 175, 141, 148, 177, 177, 148, 151, 146, 177, 186, 176, 176, 148, 143, 147, 149, 200, 172, 200, 178, 176, 181, 148, 149, 145, 178, 148, 150, 146, 178, 145, 146, 149, 148, 148, 147, 144, 144, 181, 147, 146, 181, 142, 147, 148, 172, 175, 175, 147, 175, 146, 151, 148, 145, 145, 144, 141, 150, 149, 148, 146, 200, 150, 144, 145, 149, 149, 140, 145, 173, 145, 147, 175, 149, 178, 147, 144, 148, 177, 176, 148, 181, 146]
Average steps to solution:  157.55
